In [20]:
import pymongo
import numpy as np
from pprint import pprint

# Parparing the dataset
## Inserting data into db
For windows platforms, please download the "https://www.mongodb.com/try/download/database-tools" MongoDB database tools, using following command for importing and exporting database.

```.\mongoexport -d fit3164 -c analysis_dishes -o "export_coll.json"```
```.\mongoimport -d fit3164 -c cmd_import_test --file ".\export_coll.json" --type=json```

The following command will help import this json file into MongoDB with correct data types.
```mongoimport -d <database_name> -c <collections_name> --file "<json file>mongosql.json" --type=json```



In [21]:
client = pymongo.MongoClient('localhost', 27017)  # connect to MongoClient

db = client.fit3164  # create a database
test_coll = db.cmd_import_test # first inserted data set
dish_collection = db.Dish_collection # real data set for using.

In [22]:
# pick all dish names
res = test_coll.find({})
name_coll = []
for doc in res:
    name_coll.append(doc["name"])
print(len(name_coll)) # 16857

16857


In [23]:
def converter(doc_input: dict) -> dict:
    """
    Convert "ingredient_array","NER" and "instructions" to correct format.
    :param doc_input: dict
    :return: dict
    """
    process_field = doc_input["ingredient_array"].rstrip("]")
    process_field = process_field.lstrip("[")
    process_field = process_field.split('"')

    field_p1_out = []
    for sub_str in process_field:
        # print(sub_str)
        # print(len(sub_str))
        if len(sub_str) > 2: # remove all ', ' and '' elements
            field_p1_out.append(sub_str)
    # print("-----------------")
    # print(field_p1_out)
    doc_input["ingredient_array"] = field_p1_out # the ingredient_array is now a array with strings.

    process_field_2 = doc_input["instructions"].rstrip("]")
    process_field_2 = process_field_2.lstrip("[")
    process_field_2 = process_field_2.split('"')
    # print(process_field_2)
    # print(type(process_field_2))
    field_p2_out = []
    for sub_str in process_field_2:
        # print(sub_str)
        # print(len(sub_str))
        if len(sub_str) > 2: # remove all ', ' and '' elements
            field_p2_out.append(sub_str)
    # print("-----------------")
    # print(field_p2_out)
    doc_input["instructions"] = field_p2_out

    process_field_3 = doc_input["NER"].rstrip("]")
    process_field_3 = process_field_3.lstrip("[")
    process_field_3 = process_field_3.split('"')
    # print(process_field_3)
    # print(type(process_field_2))
    field_p3_out = []
    for sub_str in process_field_3:
        # print(sub_str)
        # print(len(sub_str))
        if len(sub_str) > 2: # remove all ', ' and '' elements
            field_p3_out.append(sub_str)
    # print("-----------------")
    # print(field_p3_out)
    doc_input["NER"] = field_p3_out

    return doc_input


In [9]:
# If necessary, before next cell for inserting data
# DROP collection here:
dish_collection.drop()

In [10]:
# Run this cell at the end of the whole file.
# Insert the new                                      data into database
count = 0
for dish_name in name_coll:
    res = test_coll.find({"name": dish_name})

    temp = []
    for i in res:
        temp.append(i)

    res_update = converter(temp[0]) # a new dict with correct data type.
    ## if necessary, update the vector here then insert into mongodb ##

    del res_update['_id'] # one db can not have 2 same _id field docs.
    # pprint(res_update)
    result = dish_collection.insert_one(res_update)
    print("----------------------------------------dish insert: " + str(count))
    count += 1

----------------------------------------dish insert: 0
----------------------------------------dish insert: 1
----------------------------------------dish insert: 2
----------------------------------------dish insert: 3
----------------------------------------dish insert: 4
----------------------------------------dish insert: 5
----------------------------------------dish insert: 6
----------------------------------------dish insert: 7
----------------------------------------dish insert: 8
----------------------------------------dish insert: 9
----------------------------------------dish insert: 10
----------------------------------------dish insert: 11
----------------------------------------dish insert: 12
----------------------------------------dish insert: 13
----------------------------------------dish insert: 14
----------------------------------------dish insert: 15
----------------------------------------dish insert: 16
----------------------------------------dish insert: 17
--

# Start trying do word embedding and faiss with a smaller set of data.

In [24]:
# Pick all unique NER as dictionary
cursor = dish_collection.find({})
NER_collector = []
for doc in cursor:
    NER_collector = NER_collector + doc["NER"]
print(len(NER_collector))

157471


In [ ]:
# Using np.unique:
NER_collector = np.array(NER_collector)
NER_dic = np.unique(NER_collector)
# print(NER_dic)
NER_dic_array = []
for i in NER_dic:
    NER_dic_array.append(i)


print(len(NER_dic_array))

# Some NER is wrong:
"""
'S\u00ae
's Cookbook
's cheese
's margarine
's meat
's salsa
's sugar
(Texturized
, Blanched
, Store
, Washed
. wheat
/ Japanese sweet rice wine
/ Sauce
/Fajita Super
/Jamon Ham
\\u00b0 # unicode for degree, this by default is F degree
"""
# Need to be handly fixed

In [35]:
# same that dict in a txt file for saving time.
def save_txt():
    file = open("NER_dic.txt", 'w')
    file.write(str(NER_dic_array))# write the array into a txt file.
    file.close

save_txt()

In [41]:
def read_txt():
    file = open("NER_dic.txt", encoding= "utf-8")
    txt = file.read()
    txt_out = eval(txt)
    file.close()
    return txt_out
NER_dic = read_txt()
print(type(NER_dic))
print(len(NER_dic))
NER_dic

<class 'list'>
10504


['A Microplane',
 'A large',
 'AIOLI',
 'APPLE FILLING',
 'Accent',
 'Accent seasoning',
 'Accompaniment',
 'Accompaniments',
 'Achiote Oil',
 'Achiote Powder',
 'Active Dry Yeast',
 'Active Yeast',
 'Adobo Powder',
 'Adobo Sauce',
 "Advocado's",
 'Agedashi Syrup',
 'Alaskan king crab',
 'Alfredo Sauce',
 'Alfredo sauce',
 'Alfredo sauce I',
 'Alfredo sauce mix',
 'All Spice',
 'All-Bran',
 'All-Purpose Flour',
 'All-purpose',
 'All-purpose Flour',
 'Allegro meat marinade',
 'Allspice',
 'Almond Breeze Vanilla',
 'Almond Butter',
 'Almond Extract',
 'Almond Milk',
 'Almond slices',
 'Almonds',
 'Aluminum Foil',
 'Amarillo-Pineapple Salsa',
 'American cheese',
 'Anaheim chile',
 'Anaheim chiles',
 'Anaheim pepper',
 'Ancho Chili Powder',
 'Ancho-Guajillo Chile Sauce',
 'And Cheese',
 'Andouille sausage',
 'Angel',
 'Anise',
 'Anise Seeds',
 'Antipasto',
 'Apple',
 'Apple Cider',
 'Apple Cider Vinegar',
 'Apple cider vinegar',
 'Apple peel',
 'Apple raita',
 'Apple slices',
 'Apples',
 '

# Now, dictionary is ready, database is ready, start dong word embedding.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim